# Import Libraries

In [19]:
import pandas as pd
import numpy as np
from hmmlearn import hmm
from hmmlearn.hmm import GaussianHMM
from sklearn.decomposition import PCA as sklearnPCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
import os
import math
import time
import multiprocessing

# Principal Component Analysis of Left,Right,Head

In [11]:
X=[[521, 106], [511, 288], [364, 296], [251, 490], [487, 346], [668, 288], [660, 510], [592, 390], [430, 597], [585, 601], [485, 84], [549, 82], [434, 113], [589, 111]]
for x in range(len(X)):
    if(X[x]==None):
        X[x]=[1000,1000]
pca = sklearnPCA(n_components=1)
X=pca.fit_transform(X)
#pca.explained_variance_ratio_ #2% information loss
X

array([[-201.64004003],
       [ -19.37493551],
       [  -1.83578616],
       [ 199.09968628],
       [  40.06255713],
       [ -29.5810794 ],
       [ 192.46940212],
       [  77.14372284],
       [ 294.23705456],
       [ 288.15246439],
       [-221.2532431 ],
       [-227.40947898],
       [-188.99921262],
       [-201.07111152]])

In [12]:
left=[[156, 208], None, None, None, None, None, [308, 191], [300, 265], [293, 269], None, [284, 243], [272, 262], [268, 267], [260, 218], [259, 238], [251, 259], [245, 268], [250, 219], [243, 235], [236, 245], [227, 257]]
for x in range(len(left)):
    if(left[x]==None):
        left[x]=[0,0]
pca = sklearnPCA(n_components=1)
left=pca.fit_transform(left)
#pca.explained_variance_ratio_ 15% information loss

In [13]:
right=[[15, 180], [33, 178], [65, 172], [99, 169], [131, 161], [75, 123], [122, 130], None, None, [90, 109], [153, 129], None, None, [106, 106], [152, 130], None, None, [113, 109], [144, 130], [140, 138], None]
for x in range(len(right)):
    if(right[x]==None):
        right[x]=[0,0]
pca = sklearnPCA(n_components=1)
right=pca.fit_transform(right)
#pca.explained_variance_ratio_ #15% information loss

In [14]:
dfhead=pd.DataFrame(data=X)
dfhead=dfhead.T
dfhead=dfhead.rename(columns={0:"head_1",1:"head_2",2:"head_3",3:"head_4",4:"head_5",5:"head_6",6:"head_7",7:"head_8",8:"head_9",9:"head_10",10:"head_11",11:"head_12",12:"head_13",13:"head_14"})
dfhead

,head_1,head_2,head_3,head_4,head_5,head_6,head_7,head_8,head_9,head_10,head_11,head_12,head_13,head_14
0,-201.64004,-19.374936,-1.835786,199.099686,40.062557,-29.581079,192.469402,77.143723,294.237055,288.152464,-221.253243,-227.409479,-188.999213,-201.071112


# Convert all values into a Pandas DataFrame

#### Transform Left Hand keypoints to a Pandas DataFrame

In [16]:
dfleft=pd.DataFrame(data=left)
dfleft=dfleft.T
dfleft=dfleft.rename(columns={0:"left_1",1:"left_2",2:"left_3",3:"left_4",4:"left_5",5:"left_6",6:"left_7",7:"left_8",8:"left_9",9:"left_10",10:"left_11",11:"left_12",12:"left_13",13:"left_14",14:"left_15",15:"left_16",16:"left_17",17:"left_18",18:"left_19",19:"left_20",20:"left_21"})
dfleft

,left_1,left_2,left_3,left_4,left_5,left_6,left_7,left_8,left_9,left_10,...,left_12,left_13,left_14,left_15,left_16,left_17,left_18,left_19,left_20,left_21
0,-3.358081,252.489634,252.489634,252.489634,252.489634,252.489634,-103.179776,-147.668362,-145.260297,252.489634,...,-125.1081,-125.57892,-86.375717,-99.251317,-107.677588,-109.404449,-79.727924,-85.484911,-87.159372,-88.729032


### Transform Right Hand Keypoints to a Pandas DataFrame

In [17]:
dfright=pd.DataFrame(data=right)
dfright=dfright.T
dfright=dfright.rename(columns={0:"right_1",1:"right_2",2:"right_3",3:"right_4",4:"right_5",5:"right_6",6:"right_7",7:"right_8",8:"right_9",9:"right_10",10:"right_11",11:"right_12",12:"right_13",13:"right_14",14:"right_15",15:"right_16",16:"right_17",17:"right_18",18:"right_19",19:"right_20",20:"right_21"})
dfright

,right_1,right_2,right_3,right_4,right_5,right_6,right_7,right_8,right_9,right_10,...,right_12,right_13,right_14,right_15,right_16,right_17,right_18,right_19,right_20,right_21
0,-34.467756,-44.094899,-59.295499,-78.089206,-91.723543,-27.140608,-61.849758,115.823782,115.823782,-25.504603,...,115.823782,115.823782,-33.104903,-80.505436,115.823782,115.823782,-39.80729,-75.530589,-79.308218,115.823782


### Concatenate the head,left,right dataframe and save it to a csv.

In [18]:
df=pd.concat([dfhead, dfleft,dfright], axis=1)
df=df[0:0]
df.to_csv("sample.csv",index=False)

In [20]:
import os

def buildGestureDict(path):
	if os.path.isdir(path):
		gestureDict = {}

		for item in os.listdir(path):
			subpath = os.path.join(path, item)
			if (os.path.isdir(subpath)):
				gesture = gestureDict[item] = {
					'videos': []
				}

				for item in os.listdir(subpath):
					if (item.lower().endswith('.mp4')):
						split = item.split('-')
						video = {
							'filepath': subpath+"/"+item, # with respect to the path argument
							'filename': item,
							'no': int(split[0]),
							'gesture': split[1],
							'actor': split[2].split('.')[0]
						}

						gesture['videos'].append(video)

		return gestureDict
    
sample=buildGestureDict("dataset/")
sample

{'1': {'videos': [{'filepath': 'dataset/1/1-1-jer.mp4',
    'filename': '1-1-jer.mp4',
    'no': 1,
    'gesture': '1',
    'actor': 'jer'},
   {'filepath': 'dataset/1/1-1-jolo.mp4',
    'filename': '1-1-jolo.mp4',
    'no': 1,
    'gesture': '1',
    'actor': 'jolo'},
   {'filepath': 'dataset/1/1-1-kyle.mp4',
    'filename': '1-1-kyle.mp4',
    'no': 1,
    'gesture': '1',
    'actor': 'kyle'},
   {'filepath': 'dataset/1/1-1-min.mp4',
    'filename': '1-1-min.mp4',
    'no': 1,
    'gesture': '1',
    'actor': 'min'}]},
 '2': {'videos': [{'filepath': 'dataset/2/1-2-jer.mp4',
    'filename': '1-2-jer.mp4',
    'no': 1,
    'gesture': '2',
    'actor': 'jer'},
   {'filepath': 'dataset/2/1-2-jolo.mp4',
    'filename': '1-2-jolo.mp4',
    'no': 1,
    'gesture': '2',
    'actor': 'jolo'},
   {'filepath': 'dataset/2/1-2-kyle.mp4',
    'filename': '1-2-kyle.mp4',
    'no': 1,
    'gesture': '2',
    'actor': 'kyle'},
   {'filepath': 'dataset/2/1-2-min.mp4',
    'filename': '1-2-min.mp4',
  

In [30]:
for x in sample:
    print(x+".csv")
    for video in  sample[x]["videos"]:
        filepath=video["filepath"]
        gesture=video["gesture"]
        speaker=video["actor"]

again.csv
all.csv
bad.csv
bathroom.csv
before.csv
big.csv
boy.csv
doctor.csv
father.csv
fine.csv
finish.csv
first.csv
friday.csv
friend.csv
girl.csv
goodbye.csv
good_1.csv
good_2.csv
hello.csv
help.csv
hospital.csv
hungry.csv
language.csv
later.csv
live.csv
love.csv
meet_1.csv
meet_2.csv
monday.csv
morning.csv
mother.csv
name.csv
night.csv
no.csv
nothing.csv
okay.csv
please.csv
ready_1.csv
ready_2.csv
saturday.csv
small.csv
sorry.csv
sunday.csv
teach.csv
thank you.csv
thankyou.csv
thursday.csv
today.csv
tomorrow.csv
tuesday.csv
understand.csv
wednesday.csv
welcome.csv
where.csv
yes.csv
you.csv


In [31]:
testing=pd.read_csv("testsave.csv")
testing

,head_1,head_2,head_3,head_4,head_5,head_6,head_7,head_8,head_9,head_10,...,right_15,right_16,right_17,right_18,right_19,right_20,right_21,frame,gesture,speaker
0,-45.598327,-150.024087,-12.268116,-123.150291,592.648717,-286.669179,-480.161243,-545.157230,592.648717,592.648717,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,Monday,Kalbo
1,153.183665,-9.134377,72.792640,-77.131622,-250.813036,-93.632597,-350.285875,-458.781987,-283.665699,625.047642,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,Monday,Kalbo
2,-257.252334,-62.034792,-59.908610,164.991826,228.300128,-60.082127,164.743944,357.580747,300.818789,306.944927,...,-28.219854,-34.746535,-37.371310,-19.540587,-19.138907,-23.176291,-27.011647,2.0,Monday,Kalbo
3,-5.939419,-101.366557,28.979405,-80.408256,-2.586001,-235.785831,-408.979006,-524.054760,635.476964,635.476964,...,-37.923946,-44.914255,-51.813803,2.043373,-14.900132,-21.544593,-26.038005,3.0,Monday,Kalbo
4,112.110388,-22.031792,80.281248,-120.040494,-35.864636,-123.031054,-337.083639,-487.810207,-258.550737,653.333624,...,-143.039737,-164.878824,-183.784952,-101.583447,-115.080412,-131.672631,-148.637956,4.0,Monday,Kalbo
5,108.081296,-20.421535,80.014678,-107.188126,-46.280841,-125.197584,-343.226732,-481.813832,-250.757951,655.659977,...,6.305895,-24.804994,-44.315058,44.880166,6.849808,-14.211785,-33.008497,5.0,Monday,Kalbo
6,-3.276345,-95.210267,39.666116,-99.679610,-36.700066,-231.332563,-407.601902,-520.696332,641.482070,641.482070,...,-184.234591,-209.353297,-225.163499,-139.994216,-151.014993,-177.419943,-188.838875,6.0,Monday,Kalbo
7,114.454000,-17.861565,80.268726,-129.959591,-69.279502,-120.232053,-327.345778,-483.367255,-253.309087,654.080462,...,-192.030968,-216.702241,-236.887828,-153.989834,-162.217309,-180.907667,-200.508886,7.0,Monday,Kalbo
8,112.976129,-18.245013,79.061014,-129.868975,-53.940850,-120.187341,-336.663298,-484.366134,-252.279729,654.009422,...,-138.727956,-162.466100,-179.527890,-96.913220,-113.375622,-128.882564,-146.929486,8.0,Monday,Kalbo
9,29.456970,-80.555567,40.769113,-101.061281,644.523222,-207.410963,-392.267172,-518.445008,-256.518026,644.523222,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,9.0,Monday,Kalbo


In [32]:
path = 'gestures/'
df=pd.read_csv("sample.csv")
for filename in os.listdir(path):
    #print(filename+".csv")
    dffile=pd.read_csv("csvdata"+"/"+filename+".csv")
    df=df.append(dffile,sort=False)
df["frame"]=df["frame"].astype(int)
df.to_csv("dataset.csv",index=False)

In [2]:
data=pd.read_csv("dataset.csv")
gestures=data["gesture"].unique()
#data=data[data["speaker"]!="ling_f"]
for x in gestures:
    datapergesture=data[data["gesture"]==x]
    lenSpeakers=len(datapergesture["speaker"].unique())
    if(lenSpeakers<5):
        data=data[data["gesture"]!=x]

In [3]:
def train_all(df):
    models={}
    words=df["gesture"].unique()
    for word in words:
        dataword=df[df["gesture"]==word]
        speakers=dataword["speaker"].unique()
        lengths=[]
        for speaker in speakers:
            lengths.append(len(dataword[dataword["speaker"]==speaker]))
        dataword=dataword.drop(columns=[dataword.columns[56],dataword.columns[57],dataword.columns[58]])
        models[word]=GaussianHMM(n_components=3, n_iter=1000).fit(dataword,lengths)
    return models

In [4]:
def recognize(dictModels,data):
    probmodels={}
    prob=float("-inf")
    word=None
    data=data.drop(columns=[data.columns[56],data.columns[57],data.columns[58]])
    for modelword,model in dictModels.items():
        try:
            modelprob=model.score(data)
            probmodels[modelword]=modelprob
            if(modelprob>prob):
                prob=modelprob
                word=modelword
        except Exception as e:
            continue
    #print(probmodels)
    return word

## 60%-40% Testing

In [7]:
training={}
words=data["gesture"].unique()
totalTrue=0
totalTest=0
start=time.time()
for word in words:
    true=0
    traininglength=0
    testlength=0
    dataperword=data[data["gesture"]==word]
    speakers=dataperword["speaker"].unique().tolist()
    datalength=len(speakers)
    if(datalength%2==0):
        traininglength=math.floor((datalength*.0))+1
        testlength=round((datalength*.40))
    else:
        traininglength=math.floor((datalength*.60))+1
        testlength=round((datalength*.40))-1
    trainingspeaker=speakers[0:traininglength]
    testspeakers=speakers[datalength-testlength:datalength]
    totalTest=totalTest+len(testspeakers)
    trainingdata=data[data["gesture"]!=word]
    for speaker in trainingspeaker:
        trainingdata=trainingdata.append(dataperword[dataperword["speaker"]==speaker])
    training=train_all(trainingdata)
    print("Finished Training the model for:"+word)
    #print(training)
    for testspeaker in testspeakers:
        print("Recognition for "+word+" speaker is: "+testspeaker)
        guessword=recognize(training,dataperword[dataperword["speaker"]==testspeaker])
        if(guessword==word):
            true=true+1
            totalTrue=totalTrue+1
    print("Recognized "+word+":"+ str(true)+" out of "+str(len(testspeakers)))
    #print("Total Data: "+str(datalength)+" Training Set: "+str(0)+"-"+str(traininglength)+" Test Set:"+str(datalength-testlength)+"-"+str(datalength-1))
print("Total Accuracy"+str(totalTrue/totalTest))
print("Took "+str(time.time()-start)+" seconds")

Finished Training the model for:again
Recognition for again speaker is: sschool_m_5
Recognition for again speaker is: start_f_1
Recognized again:0 out of 2
Finished Training the model for:big
Recognition for big speaker is: sschool_f_4
Recognized big:0 out of 1
Finished Training the model for:boy
Recognition for boy speaker is: slstud_m_2
Recognition for boy speaker is: sschool_m_5
Recognition for boy speaker is: start_f_1
Recognized boy:0 out of 3
Finished Training the model for:doctor
Recognition for doctor speaker is: sschool_f_5
Recognition for doctor speaker is: start_f_1
Recognized doctor:0 out of 2
Finished Training the model for:father
Recognition for father speaker is: start_f_1
Recognized father:0 out of 1
Finished Training the model for:fine
Recognition for fine speaker is: slstud_m_2
Recognition for fine speaker is: sschool_m_4
Recognized fine:0 out of 2
Finished Training the model for:friday
Recognition for friday speaker is: sschool_f_2
Recognized friday:0 out of 1
Finish

## Cross-Validation

In [8]:
training={}
words=data["gesture"].unique()
totalTrue=0
totalTest=0
start=time.time()
for word in words:
    true=0
    traininglength=0
    testlength=0
    dataperword=data[data["gesture"]==word]
    speakers=dataperword["speaker"].unique().tolist()
    datalength=len(speakers)
    totalTest=totalTest+datalength
    for index in range(datalength):
        trainingdata=data[data["gesture"]!=word]
        trainingdata=trainingdata.append(dataperword[dataperword["speaker"]!=speakers[index]])
        training=train_all(trainingdata)
        guessword=recognize(training,dataperword[dataperword["speaker"]==speakers[index]])
        if(guessword==word):
            true=true+1
            totalTrue=totalTrue+1
    print("Recognized "+word+":"+ str(true)+" out of "+str(len(speakers)))
    #print("Total Data: "+str(datalength)+" Training Set: "+str(0)+"-"+str(traininglength)+" Test Set:"+str(datalength-testlength)+"-"+str(datalength-1))
print("Total Accuracy"+str(totalTrue/totalTest))
print("Took "+str(time.time()-start)+" seconds")

Recognized again:0 out of 6
Recognized big:0 out of 5
Recognized boy:0 out of 8
Recognized doctor:0 out of 6
Recognized father:0 out of 5
Recognized fine:0 out of 7
Recognized friday:0 out of 5
Recognized friend:3 out of 8
Recognized good_2:0 out of 5
Recognized hello:0 out of 6
Recognized hospital:0 out of 5
Recognized hungry:0 out of 5
Recognized language:0 out of 5
Recognized live:1 out of 5
Recognized love:0 out of 6
Recognized morning:0 out of 5
Recognized mother:0 out of 5
Recognized name:0 out of 5
Recognized night:0 out of 6
Recognized no:0 out of 7
Recognized please:0 out of 7
Recognized saturday:0 out of 5
Recognized small:0 out of 6
Recognized sorry:3 out of 7
Recognized teach:0 out of 6
Recognized thank you:0 out of 6
Recognized tomorrow:0 out of 7
Recognized understand:2 out of 7
Recognized wednesday:0 out of 6
Recognized where:2 out of 8
Recognized yes:0 out of 9
Recognized you:0 out of 7
Total Accuracy0.05612244897959184
Took 965.3825385570526 seconds


## Cross Validation with MultiProcessing

In [4]:
def cross_validate(word):
    data=pd.read_csv("dataset.csv")
    true=0
    traininglength=0
    testlength=0
    dataperword=data[data["gesture"]==word]
    speakers=dataperword["speaker"].unique().tolist()
    datalength=len(speakers)
    totalTest=totalTest+datalength
    for index in range(datalength):
        trainingdata=data[data["gesture"]!=word]
        trainingdata=trainingdata.append(dataperword[dataperword["speaker"]!=speakers[index]])
        training=train_all(trainingdata)
        guessword=recognize(training,dataperword[dataperword["speaker"]==speakers[index]])
        if(guessword==word):
            true=true+1
    print("Recognized "+word+":"+ str(true)+" out of "+str(len(speakers)))
    #print("Total Data: "+str(datalength)+" Training Set: "+str(0)+"-"+str(traininglength)+" Test Set:"+str(datalength-testlength)+"-"+str(datalength-1))


In [ ]:
training={}
words=data["gesture"].unique().tolist()
totalTrue=0
totalTest=0
starttime = time.time()
pool = multiprocessing.Pool()
pool.map(cross_validate,words)
pool.close()
print('That took {} seconds'.format(time.time() - starttime))
#print("Total Accuracy"+str(totalTrue/totalTest))

In [ ]:
import time
import multiprocessing 

def basic_func(x):
    if x == 0:
        return 'zero'
    elif x%2 == 0:
        return 'even'
    else:
        return 'odd'

def multiprocessing_func(x):
    y = x*x
    #time.sleep(2)
    print('{} squared results in a/an {} number'.format(x, basic_func(y)))
    
    
def main():
    
    starttime = time.time()
    pool = multiprocessing.Pool()
    pool.map(multiprocessing_func, range(0,10))
    pool.close()
    print('That took {} seconds'.format(time.time() - starttime))
    
main()